In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import random

In [2]:
p=0.4
df = pd.read_csv('env/datasets/cleaned_articles1.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)

df2 = pd.read_csv('env/datasets/cleaned_articles2.csv',header=None,skiprows=lambda i: 1>0 and random.random() > p)


In [3]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",after bullet shell get count blood dri votiv c...
1,4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea leader kim said ...
2,6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0,NaN,BEIJING — President Tsai of Taiwan sharpl...,beij presid tsai taiwan sharpli critic china l...
3,7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz...",danni cahil stood slight daze blizzard confett...
4,8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ...",just hillari kerr founder digit media compani ...


In [10]:
stemmer = SnowballStemmer('english')
words = stopwords.words("english")

In [11]:
df.columns = ['NaN', 'Unnamed', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content', 'cleaned']
df2.columns = ['NaN', 'Unnamed', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content', 'cleaned']

In [12]:
df[df.publication == 'Breitbart' & df.Unnamed == '']['cleaned']

TypeError: cannot compare a dtyped [int64] array with a scalar of type [bool]

In [13]:
#Second batch:
n_s_breitbart = df[df.publication == 'Breitbart']
n_s_times = df[df.publication == 'New York Times']
#First batch:
n_s_atlantic = df2[df2.publication == 'Atlantic']
n_s_post = df2[df2.publication == 'New York Post']

In [14]:
n_s = list(n_s_breitbart.iloc[:,11].values) + list(n_s_post.iloc[:,11].values) \
 + list(n_s_atlantic.iloc[:,11].values) + list(n_s_times.iloc[:,11].values)

In [15]:
n_s_breitbart.dropna()

,NaN,Unnamed,id,title,publication,author,date,year,month,url,content,cleaned


In [24]:
n_s_times.iloc[27]["cleaned"]

'gov andrew m cuomo new york said wednesday complet overhaul la guardia airport underway want rebuild new york citi airport john f kennedi intern mr cuomo outlin plan spend billion modern kennedi termin improv highway transit system connect airport he provid timet plan say specif money would come the next step tackl j f k la guardia enough mr cuomo said speak meet associ better new york busi group manhattan we need build new airport j f k go process la guardia in promot big infrastructur project includ second avenu subway open sunday mr cuomo build legaci new york perhap rais nation profil the plan offer mr cuomo wednesday suggest intend stop tri impos port author new york new jersey oper la guardia j f k the author commission schedul meet thursday vote billion spend plan includ billion improv j f k when mr cuomo form advisori group studi improv j f k said panel would make recommend author but author chairman john j degnan see panel final report tuesday we await opportun review detail 

In [17]:
n_s = [word.replace('New York Post','') for word in n_s]
n_s = [word.replace('Breitbart','') for word in n_s]
n_s = [word.replace('New York Times','') for word in n_s]
n_s = [word.replace('Atlantic','') for word in n_s]

In [18]:
classes_Bias = np.asarray([1 for i in range(len(n_s_breitbart))] + \
[1 for i in range(len(n_s_post))] + [2 for i in range(len(n_s_atlantic))] + \
[2 for i in range(len(n_s_times))])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(n_s, classes_Bias, test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [20]:
model = pipeline.fit(X_train, y_train)

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

In [ ]:
print("accuracy score: " + str(model.score(X_test, y_test)))

In [ ]:
n_s_atlantic.content[7]

In [34]:
print(model.predict(['They were primed to become eukaryotes. ” This debate is likely to rage on for a while, because no one has actually seen the Asgardians themselves. In the 1980s, scientists developed ways of identifying microbes by searching for their DNA amid environmental samples. This is now the main way of finding new species, but it means that many microbes —  like Loki and its kin —  are only known through their DNA. No one knows what they look like. “It’s high on our priority list,” says Ettema. “It’s crucial that we have a look at the cells to see what they’re doing, but that’s extremely hard. Loki was discovered in   ocean sediments, where you don’t go every day. Thor and Heimdall are present in shallower environments, but they’re less than 0. 1 percent of the total microbial community. It’s like looking for a needle in a haystack, but we’re working on it.']))

[2]


In [3]:
df3 = pd.read_pickle('cleaned_articles1.pkl')

In [4]:
df3.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,cleaned
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,washington congression republican new fear com...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...",after bullet shell get count blood dri votiv c...
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",when walt disney bambi open critic prais spare...
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...",death may great equal necessarili evenhand of ...
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...",seoul south korea north korea leader kim said ...
